In [35]:
from logic import * #logic 파일을 사용하기 위해 import 함
from inference import *
import itertools


In [36]:

def create_dormitory_kb(student_info):
    student, status, test_result, vaccinated, submitted_docs, tuition_paid, grade, distance = student_info
    clauses = []
    
    # 학생의 상태에 따라 등록 여부를 결정
    if status == '재학생':
        clauses.append(expr(f"Enroll({student})"))
    elif status == '대학원생':
        clauses.append(expr(f"Graduate({student})"))
        clauses.append(expr("Graduate(x) ==> Enroll(x)"))
    elif status == '신입생' :
        clauses.append(expr(f"Fresh({student})"))
        clauses.append(expr("Fresh(x) ==> Enroll(x)"))
    
    # 전염병 검사 결과에 따른 조건 추가
    if test_result:
        clauses.append(expr(f"E_Test({student})"))
        clauses.append(expr("E_Test(x) ==> E_testOwns(x)"))
    
    # 백신 접종 여부
    if vaccinated:
        clauses.append(expr(f"Inoculate({student})"))
    
    # 입학 서류 제출 여부
    if submitted_docs:
        clauses.append(expr(f"Addmission_docu({student}_doc)"))
        clauses.append(expr("Addmission_docu(y) ==> A_Owns(x,y)"))
        clauses.append(expr(f"A_Owns({student}, {student}_doc) ==> Submit({student}, {student}_doc)"))
    
    # 등록금 지급 여부
    if tuition_paid:
        clauses.append(expr(f"Register({student})"))
    
    # 성적 정보 (학부생의 경우)
    if grade is not None:
        clauses.append(expr(f"Grade({student})"))
    elif grade is None : 
        if 'status' == '신입생' :
           clauses.append(expr(f"Grade({student})")) 
    
    # 위치 정보
    clauses.append(expr(f"Home({distance})"))
    clauses.append(expr("Home(z) ==> Location(x,z)"))

    # 최종 조건
    clauses.append(expr("(Enroll(x) & E_testOwns(x) & Inoculate(x) & Submit(x,y) & Register(x) & Grade(x) & Location(x,z)) ==> Join(x)"))
    
    return FolKB(clauses)


In [37]:
student_A_info = ('A', '재학생', False, True, True, True, 3.2, 114)
dormitoryA_kb = create_dormitory_kb(student_A_info)

student_B_info = ('B','재학생', True, True, True, True, 3.8, 273)
dormitoryB_kb = create_dormitory_kb(student_B_info)

student_C_info = ('C', '재학생', True, True, True, True, 3.8, 27)
dormitoryC_kb = create_dormitory_kb(student_C_info)

student_D_info = ('D', '신입생', True, True, True, True, 3.8, 27)
dormitoryD_kb = create_dormitory_kb(student_D_info)

student_E_info = ('E', '대학원생', True, True, True, True, 4.3, 433)
dormitoryE_kb = create_dormitory_kb(student_E_info)

student_F_info = ('F', '대학원생', True, False, False, True, 4.2, 36)
dormitoryF_kb = create_dormitory_kb(student_F_info)


In [38]:
# dormitoryA_kb.ask는 fol_bc_ask를 호출함
pass_student = []

for letter in ['A', 'B', 'C', 'D', 'E', 'F']:
    if letter == 'A':
        if dormitoryA_kb.ask(expr('Join(x)')) != False:
            pass_student.append('A')
    elif letter == 'B':
        if dormitoryB_kb.ask(expr('Join(x)')) != False:
            pass_student.append('B')
    elif letter == 'C':
        if dormitoryC_kb.ask(expr('Join(x)')) != False:
            pass_student.append('C')
    elif letter == 'D':
        if dormitoryD_kb.ask(expr('Join(x)')) != False:
            pass_student.append('D')
    elif letter == 'E':
        if dormitoryE_kb.ask(expr('Join(x)')) != False:
            pass_student.append('E')
    elif letter == 'F':
        if dormitoryF_kb.ask(expr('Join(x)')) != False:
            pass_student.append('F')
print(pass_student)

['B', 'C', 'D', 'E']


In [39]:
score_hash_table = {} # 학생별 성적 정의 해쉬 테이블

score_hash_table['A'] = 3.2
score_hash_table['B'] = 3.8
score_hash_table['C'] = 3.8
score_hash_table['D'] = 0
score_hash_table['E'] = 4.3
score_hash_table['F'] = 4.2




In [40]:
distance_hash_table = {} # 학생별 학교를 기준으로 사는 위치 거리 정보 해쉬 테이블 (단위: km)

distance_hash_table['A'] = 114
distance_hash_table['B'] = 273
distance_hash_table['C'] = 27
distance_hash_table['D'] = 21
distance_hash_table['E'] = 433
distance_hash_table['F'] = 36



In [41]:

def score_cal(x): #score_cal() 함수는 주어진 값을 기반으로 성적 점수를 환산하는 함수이다.
    score1 = 0
    if x>=4.0:
        score1 += 30
    elif x>=3.0:
        score1 += 20
    #신입생의 경우 성적을 보지 않으니까 30점을 부여해 일반 재학생 보다 우선권을 줄 수 있게 한다.
    elif x == 0:
        score1 += 30
    else:
        score1 += 10
    return score1
        
def distance_cal(y): #distance_cal() 함수는 주어진 값을 기반으로 거리 점수를 환산하는 함수이다.
    score2 = 0
    if y>=120:
        score2 += 40
    elif y>= 60:
        score2 += 30
    else:
        score2 += 10
    return score2



In [42]:
# 학생별 성적 및 거리 데이터 매핑
passstudent_score_ht = {item: score_hash_table.get(item, 0) for item in pass_student}
passstudent_distance_ht = {item: distance_hash_table.get(item, 0) for item in pass_student}

# 성적과 거리에 따른 점수 계산
score_list1 = [distance_cal(y) for y in passstudent_distance_ht.values()]
score_list2 = [score_cal(x) for x in passstudent_score_ht.values()]

# 점수 합산
def add_lists(list1, list2):
    return [list1[i] + list2[i] for i in range(min(len(list1), len(list2)))]

result_list = add_lists(score_list1, score_list2)

# 최종 점수 매핑
final_ht = dict(zip(pass_student, result_list))

# 점수가 높은 순으로 학생들 정렬
sorted_hash_table = dict(sorted(final_ht.items(), key=lambda x: x[1], reverse=True))

# 결과 출력
print("Pass Students:", pass_student)
print("Final Scores:", final_ht)
print("Sorted Scores:", sorted_hash_table)

Pass Students: ['B', 'C', 'D', 'E']
Final Scores: {'B': 60, 'C': 30, 'D': 40, 'E': 70}
Sorted Scores: {'E': 70, 'B': 60, 'D': 40, 'C': 30}
